In [1]:
from dotenv import load_dotenv
import os
load_dotenv()

True

In [2]:
# Load image and convert to base64
import base64
from pathlib import Path

# Load the test image
image_path = Path("test_human.jpeg")
with open(image_path, "rb") as image_file:
    image_data = image_file.read()

# Convert to base64
image_base64 = base64.b64encode(image_data).decode("utf-8")
print(f"Image loaded successfully! Size: {len(image_data)} bytes")
print(f"Base64 length: {len(image_base64)} characters")



Image loaded successfully! Size: 1288035 bytes
Base64 length: 1717380 characters


In [3]:
type(image_base64)

str

In [4]:
# Test the API endpoint
import requests
import json

# API endpoint URL (adjust if running on different host/port)
api_url = "http://localhost:8000/fashion-workflow"

# Test data
test_data = {
    "base64_image": image_base64,  # Using the base64 from previous cell
    "user_input": "What should I wear to a job interview?"
}

# Make the API request
try:
    print("Sending request to API...")
    response = requests.post(api_url, json=test_data)
    
    print(f"Status Code: {response.status_code}")
    print(f"Response Headers: {dict(response.headers)}")
    
    if response.status_code == 200:
        result = response.json()
        print("✅ API call successful!")
        print(f"Success: {result.get('success')}")
        print(f"Text Response: {result.get('text', 'No text')[:200]}...")
        print(f"Number of images: {len(result.get('images', []))}")
        
        if result.get('error_message'):
            print(f"Error message: {result.get('error_message')}")
    else:
        print(f"❌ API call failed with status {response.status_code}")
        print(f"Response: {response.text}")
        
except requests.exceptions.ConnectionError:
    print("❌ Connection failed. Make sure the API server is running on localhost:8000")
    print("Start the server with: python api.py")
except Exception as e:
    print(f"❌ Error: {str(e)}")


Sending request to API...
Status Code: 200
Response Headers: {'date': 'Sat, 11 Oct 2025 14:13:53 GMT', 'server': 'uvicorn', 'content-length': '782', 'content-type': 'application/json'}
✅ API call successful!
Success: True
Text Response: This season, effortless power and understated elegance are key. We're seeing a beautiful duality emerge: the first look embodies quiet confidence with a crisp navy pantsuit, softened by a silk blouse ...
Number of images: 2


In [ ]:
# Debug: Let's see the actual structure of the result
print("Full result structure:")
print(json.dumps(result, indent=2))

# Visualize the first image in the result (if present)
from PIL import Image
import io
import base64
import matplotlib.pyplot as plt

images = result.get("images", [])
print(f"\nNumber of images: {len(images)}")

if images:
    print(f"First image structure: {images[0]}")
    
    # Try different possible keys for the base64 data
    first_img = None
    possible_keys = ["base64", "image", "data", "content"]
    
    for key in possible_keys:
        if key in images[0]:
            first_img = images[0][key]
            print(f"Found image data under key: {key}")
            break
    
    if first_img:
        try:
            img_bytes = base64.b64decode(first_img)
            img = Image.open(io.BytesIO(img_bytes))
            plt.figure(figsize=(10, 8))
            plt.imshow(img)
            plt.axis('off')
            plt.title(images[0].get("description", "Generated Image"))
            plt.show()
        except Exception as e:
            print(f"Error displaying image: {e}")
    else:
        print("No base64 image data found in the first image.")
        print("Available keys in first image:", list(images[0].keys()))
else:
    print("No images found in result.")


No base64 image data found in the first image.
